In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' 
import numpy as np
import math
import tensorflow as tf
import sys
import h5py
from pylab import *
from libs.utils import weight_variable, bias_variable
from libs.readfile_fau import readFile, separa_fau, combine_five
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


Using TensorFlow backend.


In [2]:
 def batch_norm(x,train=True,epsilon=1e-5,momentum=0.9):
        return tf.contrib.layers.batch_norm(x,
                          decay=momentum, 
                          updates_collections=None,
                          epsilon=epsilon,
                          scale=True,
                          is_training=train)

In [3]:
#load the train and test data
filename_train = './data/CS_Ftrain_nor.arff'
filename_test =  './data/CS_Ftest_nor.arff'
train_data, train_label, train_size = readFile(filename_train)
test_data, test_label, test_size = readFile(filename_test)
print("Train data：{0} , {1} , {2}".format(train_data.shape, train_label.shape, train_size))
print("Test data：{0} , {1} , {2}".format(test_data.shape, test_label.shape, test_size))
fau_a,fau_e,fau_n,fau_p,fau_r = separa_fau(train_data, train_label)
print('A: {0} , E: {1} , N: {2} , P: {3} , R: {4}'.format(fau_a.shape, fau_e.shape, fau_n.shape, fau_p.shape, fau_r.shape))

combine = [fau_a,fau_e,fau_n,fau_p,fau_r]
emotion = ['Anger', 'Emphatic', 'Neutral', 'Positive', 'Rest']
fake_emotion_txt = ['fake_a.txt', 'fake_e.txt', 'fake_n.txt', 'fake_p.txt', 'fake_r.txt']

d_dim = 384
d_layer1 = 500
d_layer2 = 300
d_layer3 = 200
#d_layer4 = 200

g_dim = 100
g_layer1 = 300
g_layer2 = 200
g_layer3 = 100
#g_layer4 = 100

x_d = tf.placeholder(tf.float32, shape = [None, d_dim])
x_g = tf.placeholder(tf.float32, shape = [None, g_dim])

keep_prob = 0.5
learning_rate = 0.0002
fake_num = 2000

weights = {
    'w_d1' : weight_variable([d_dim, d_layer1], 'w_d1'),
    'w_d2' : weight_variable([d_layer1, d_layer2], 'w_d2'),
    'w_d3' : weight_variable([d_layer2, d_layer3], 'w_d3'),
    'w_d4' : weight_variable([d_layer3, 1], 'w_d4'),
    
    'w_g1' :  weight_variable([g_dim, g_layer1], 'w_g1'),
    'w_g2' :  weight_variable([g_layer1, g_layer2], 'w_g2'),
    'w_g3' :  weight_variable([g_layer2, g_layer3], 'w_g3'),
    'w_g4' : weight_variable([g_layer3, d_dim], 'w_g4'),
}
      
biases = {
    'b_d1' : bias_variable([d_layer1], 'b_d1'),
    'b_d2' : bias_variable([d_layer2], 'b_d2'),
    'b_d3' : bias_variable([d_layer3], 'b_d3'),
    'b_d4' : bias_variable([1], 'b_d4'),
                          
    'b_g1' : bias_variable([g_layer1], 'b_g1'),
    'b_g2' : bias_variable([g_layer2], 'b_g2'),
    'b_g3' : bias_variable([g_layer3], 'b_g3'),
    'b_g4' : bias_variable([d_dim], 'b_g4'),
}
      
var_d = [weights['w_d1'], weights['w_d2'], weights['w_d3'],weights['w_d4'],
              biases['b_d1'], biases['b_d2'], biases['b_d3'], biases['b_d4']]
var_g = [weights['w_g1'], weights['w_g2'], weights['w_g3'],weights['w_g4'],
              biases['b_g1'], biases['b_g2'], biases['b_g3'], biases['b_g4']]


def onehot(test_label):
    onehot=[]
    for i in test_label:
        if i == 0:
            label=[1,0,0,0,0]
        if i == 1:
            label=[0,1,0,0,0]
        if i == 2:
            label=[0,0,1,0,0]
        if i == 3:
            label=[0,0,0,1,0]
        if i == 4:
            label=[0,0,0,0,1]
        onehot.append(label)
    return onehot

# leaky relu activation function
def lrelu(x, leak=0.2, name="lrelu"):
      return tf.maximum(x, leak*x)

Train data：(9959, 384) , (9959, 5) , [881, 2093, 5590, 674, 721]
Test data：(8257, 384) , (8257, 5) , [611, 1508, 5377, 215, 546]
A: (881, 384) , E: (2093, 384) , N: (5590, 384) , P: (674, 384) , R: (721, 384)


In [8]:
### GAN block ###
def generator(z):
    # use leaky relu as activation function except output layer
    h_g1 = lrelu(tf.matmul(z, weights['w_g1']) + biases['b_g1'])
    h_g2 = lrelu(tf.matmul(h_g1, weights['w_g2']) + biases['b_g2'])
    h_g3 = lrelu(tf.matmul(h_g2, weights['w_g3']) + biases['b_g3'])
    h_g4 = lrelu(tf.matmul(h_g3, weights['w_g4']) + biases['b_g4'])
    #h_g5 = tflearn.activations.leaky_relu(tf.matmul(h_g4, weights['w_g5']) + biases['b_g5'], alpha=0.2)
    # ouptut layer use tanh as activation function
    y_g = tf.nn.tanh(h_g4) 
    return y_g

def discriminator(x):
     # use leaky relu as activation function except output layer
    h_d1 = lrelu(tf.matmul(x, weights['w_d1']) + biases['b_d1'])
    h_d2 = lrelu(tf.matmul(h_d1, weights['w_d2']) + biases['b_d2'])
    h_d3 = lrelu(tf.matmul(h_d2, weights['w_d3']) + biases['b_d3'])
    h_d4 = lrelu(tf.matmul(h_d3, weights['w_d4']) + biases['b_d4'])
    #h_d5 = tf.nn.dropout(tflearn.activations.leaky_relu(tf.matmul(h_d4, weights['w_d5']) + biases['b_d5'], alpha=0.2), keep_prob)
    # ouptut layer use sigmoid as activation function
    y_d = tf.nn.sigmoid(h_d4)
    return y_d

def noise_sample(m, n):
    return np.random.normal(0.0, 1.0, size=[m, n]) # use gaussian distribution to generate noise sample (mean=0, stddev=1)

def plot(num, samples):
    samples = np.concatenate((samples,np.zeros((samples.shape[0],16))),axis=1)
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(20, 20), cmap = 'gray')
    plt.savefig('./pic/figure %d .png'%(num))
    #plt.show()
    
def train_gan(emotion_class, emotion_name):
    gan_train_data = emotion_class # set the emotion class
    g_sample = generator(x_g)
    d_real= discriminator(x_d)
    d_fake = discriminator(g_sample)
    
    d_loss = -tf.reduce_mean(tf.log(d_real) + tf.log(1. - d_fake))
    g_loss = -tf.reduce_mean(tf.log(d_fake))
    
    global_step = tf.Variable(0)
    # update discriminator
    learning_rate_d = tf.train.exponential_decay(learning_rate, decay_steps=10000, decay_rate=0.5, global_step=global_step)
    d_optimizer = tf.train.GradientDescentOptimizer(learning_rate_d).minimize(d_loss, var_list = var_d)# use SGD optimizer, minimize 
    
    # update generator
    learning_rate_g = tf.train.exponential_decay(learning_rate,decay_steps=10000, decay_rate=0.5,global_step=global_step)
    g_optimizer = tf.train.AdamOptimizer(learning_rate_g).minimize(g_loss, var_list = var_g)# use Adam optimizer, maximize logD
    
    num = 0
    batch_size = 100
    global_step = 30001
    sess = tf.InteractiveSession()
    init_op = tf.global_variables_initializer()
    saver = tf.train.Saver({ 'w_g1': weights['w_g1'], 'w_g2': weights['w_g2'], 'w_g3': weights['w_g3'], 'w_g4': weights['w_g4'],
                                           'w_d1': weights['w_d1'], 'w_d2': weights['w_d2'], 'w_d3': weights['w_d3'], 'w_d4': weights['w_d4'],
                                           'b_g1': biases['b_g1'], 'b_g2': biases['b_g2'], 'b_g3': biases['b_g3'], 'b_g4': biases['b_g4'],
                                           'b_d1': biases['b_d1'], 'b_d2': biases['b_d2'], 'b_d3': biases['b_d3'], 'b_d4': biases['b_d4']}, max_to_keep=0)
    sess.run(init_op)
    for step in range(global_step):
        offset = (step * batch_size) % (gan_train_data.shape[0] - batch_size)
        batch_x = gan_train_data[offset:(offset + batch_size), :]
        _ , d_loss_train = sess.run([d_optimizer, d_loss], feed_dict = {x_d: batch_x, x_g: noise_sample(batch_size, g_dim)})
        _ , g_loss_train = sess.run([g_optimizer, g_loss], feed_dict = {x_g: noise_sample(batch_size, g_dim)})
        if step % 2000 == 0:
            print("Step %d, discriminator loss %.5f , generator loss %.5f" % (step, d_loss_train, g_loss_train))
        if step % 1000 == 0: 
            g_sample_plot = g_sample.eval(feed_dict = {x_g: noise_sample(16, g_dim)})
            plot(num,g_sample_plot)
            path_name = './fake_model/' + emotion_name
            save_path = saver.save(sess, path_name + '/model%d.ckpt'% (num))
            num+=1
            
def generate_fake_data():
    fake_emotion_list = [[], [], [], [], []]
    for i in range(5):
        fake_emotion_model = './fake_model/' + emotion[i] + '/model20.ckpt'
        with tf.Session() as sess:
            saver = tf.train.Saver({ 'w_g1': weights['w_g1'], 'w_g2': weights['w_g2'], 'w_g3': weights['w_g3'], 'w_g4': weights['w_g4'],
                                           'w_d1': weights['w_d1'], 'w_d2': weights['w_d2'], 'w_d3': weights['w_d3'], 'w_d4': weights['w_d4'],
                                           'b_g1': biases['b_g1'], 'b_g2': biases['b_g2'], 'b_g3': biases['b_g3'], 'b_g4': biases['b_g4'], 
                                           'b_d1': biases['b_d1'], 'b_d2': biases['b_d2'], 'b_d3': biases['b_d3'], 'b_d4': biases['b_d4']})
            saver.restore(sess,  fake_emotion_model)
            g_sample = generator(x_g)
            new_fau_sample = g_sample.eval(feed_dict = {x_g: noise_sample(fake_num, g_dim)})
           #plot(i, new_fau_sample)\n",
            fake_emotion_list[i] = new_fau_sample

    fake_a = fake_emotion_list[0]
    fake_e = fake_emotion_list[1]
    fake_n = fake_emotion_list[2]
    fake_p = fake_emotion_list[3]
    fake_r = fake_emotion_list[4]
    # save the fake data
    np.savetxt("./fake_data/Anger/fake_a.txt", fake_a)
    np.savetxt("./fake_data/Emphatic/fake_e.txt", fake_e)
    np.savetxt("./fake_data/Neutral/fake_n.txt", fake_n)
    np.savetxt("./fake_data/Positive/fake_p.txt", fake_p)
    np.savetxt("./fake_data/Rest/fake_r.txt", fake_r)
    print ('\nEnd of generating.\n')

In [11]:
def read_fake_data(filename,filename2, isSP=False) :
    
    f = open(filename, 'r')   
   
    #read data
    data = []
    label = []
    size = [0, 0, 0, 0, 0]
    
    while True:
        line = f.readline()
        if line == '' :  # EOF
            break
        
        #data processing
        temp = []
        for numstr in line.split(" ") :
            if numstr :
                numFl = numstr
                temp.append(numFl)
               
        data.append(temp)  # get data
    
    f.close()
    
    l=open(filename2,'r')    
    while True:
        line = l.readline()
        if line == '' :  # EOF
            break        
       
        temp = []
        for numstr in line.split(" ") :
            if numstr :
                numFl = numstr
                temp.append(numFl)
                
        label.append(temp)  # get label       
    
    l.close()
    
    data = np.asarray(data)
    label = np.asarray(label)
    #data, label = randomize(data, label)  #from Homework1 (Udacity)
    
    
    return data, label

In [5]:
### KNN block ###
#Euclidean distance
def euclidean(train, test, train_num):
    dist=[]
    for i in range(train_num):# number of training data: i
        dist_temp=[math.sqrt(sum(pow(train[i,:] - test, 2)))]
        dist.append(dist_temp)
    return dist

# Manhattan distance: |x1-x2|+|y1-y2|
def manhattan(train, test, train_num):
    dist=[]
    for i in range(train_num):# number of training data: i
        dist_temp=[(sum(abs(train[i,:] - test)))]
        dist.append(dist_temp)
    return dist

#Chebyshev distance:  max(|x2-x1|, |y2-y1|)
def chebyshev(train, test, train_num):
    dist=[]
    for i in range(train_num):# number of training data: i
        dist_temp=[np.max(abs((train[i,:] - test)))]
        dist.append(dist_temp)
    return dist

def knn(target_data):
    kvalue = 5 # set the k value
    a_weight=1.0/881.0
    e_weight=1.0/2093.0
    n_weight=1.0/5590.0
    p_weight=1.0/674.0
    r_weight=1.0/721.0
    dist_choice = 'eu'# distance type (eu , manha , chebyshev)
    result=[0,0,0,0,0]
    if(dist_choice == 'eu'):
        dist = euclidean(train_data, target_data, train_data.shape[0])
    if(dist_choice == 'manha'):
        dist = manhattan(train_data, target_data, train_data.shape[0])
    if(dist_choice == 'chebyshev'):
        dist = chebyshev(train_data, target_data, train_data.shape[0])

    dist_label = np.concatenate((train_label, dist), axis=1) # concatenate distance with the corresponding label
    sorted_distances = sorted(dist_label, key = lambda x : x[1]) # sort by the distance
    sorted_distances=np.asarray(sorted_distances) # convert into an array
    
    for j in range(kvalue): # record the knn result
        m = np.argmax(sorted_distances[j][0:5])
        if  m == 0: #Anger
            result[0]+=a_weight
        elif m == 1: #Emphatic
            result[1]+=e_weight
        elif m == 2: #Neutral
            result[2]+=n_weight
        elif m == 3: #Positive
            result[3]+=p_weight
        elif m == 4: #Rest
            result[4]+=r_weight
    temp = np.argmax(result)
    if temp == 0: #Anger
        return [[1,0,0,0,0]]
    elif temp == 1: #Emphatic
        return  [[0,1,0,0,0]]
    elif temp == 2: #Neutral
        return [[0,0,1,0,0]]
    elif temp == 3: #Positive
        return [[0,0,0,1,0]]
    elif temp == 4: #Rest
        return [[0,0,0,0,1]]


In [18]:
### teacher-student model block ###
def _to_tensor(x, dtype):
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

def weighted_loss(r):
    
    def new_loss(y_true, y_pred):
        _EPSILON = 10e-8
        y_pred /= tf.reduce_sum(y_pred, reduction_indices=len(y_pred.get_shape()) - 1, keep_dims=True)
        # manual computation of crossentropy
        epsilon = _to_tensor(_EPSILON, y_pred.dtype.base_dtype)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        
        #y_true = tf.cast(y_true, tf.float64)
        rr = tf.cast(r, tf.float32)
        new_R = tf.matmul(rr, y_true, transpose_b=True)
        
        return - new_R * tf.reduce_sum(y_true * tf.log(y_pred), reduction_indices=len(y_pred.get_shape()) - 1)
        #return r * K.categorical_crossentropy(y_true, y_pred)

    return new_loss

def train_stmodel(train_data,train_label):
    
    filename_test =  './data/CS_Ftest_nor.arff'
    test_data, test_label, test_size = readFile(filename_test)
    print("訓練資料：",train_data.shape,train_label.shape)
    print("測試資料：",test_data.shape,test_label.shape)
    weight_data=[]
    weight_data.append([1.1,0.5,0.2,1.5,1.4])
    #weight_data.append([1.0,1.0,1.0,1.0,1.0])
    weight_data=np.asarray(weight_data)
    
    model = Sequential()
    model.add(Dense(30, activation='sigmoid', input_shape=(384,)))
    #model.add(Dropout(0.5))
    #model.add(Dense(15, activation='sigmoid'))
    model.add(Dense(5, activation='softmax'))
    model.load_weights("./model/ts_46_model.h5")#載入原網路
    model.summary()
    

    sgd = SGD(lr=0.01, momentum=0.5, decay=0.0005)
    model.compile(loss=weighted_loss(r=weight_data), optimizer=sgd)
    history = model.fit(train_data, train_label,batch_size=100, epochs=600,verbose=2)    
    result=model.predict(train_data)
    np.save("teacher_label",result)
    
    teacher_label=np.load("teacher_label.npy")
    train_label=teacher_label#使用teacher_label
    
    model = Sequential()
    model.add(Dense(30, activation='sigmoid', input_shape=(384,)))
    #model.add(Dropout(0.5))
    #model.add(Dense(15, activation='sigmoid'))
    model.add(Dense(5, activation='softmax'))
    model.load_weights("./model/ts_46_model.h5")#載入原網路
    model.summary()
    sgd = SGD(lr=0.01, momentum=0.5, decay=0.0005)
    model.compile(loss=weighted_loss(r=weight_data), optimizer=sgd)
    history = model.fit(train_data, train_label,batch_size=100, epochs=600,verbose=2)    
    result=model.predict(test_data)
    
    AC_A = 0.0
    AC_E = 0.0
    AC_N = 0.0
    AC_P = 0.0
    AC_R = 0.0
    AC_WA = 0.0
    AC_UA = 0.0
    right=0#正確句數
    matrix = np.zeros((5, 5))

    use_data=result

    for i, (Target, Label) in enumerate( zip(test_label,use_data) ) :  ### target:答案 label:辨識結果
        m = np.max(Label)#辨識結果（最大類別）
        for j, value in enumerate(Label) :  ### 分到哪類
            if value == m :
                for k, num in enumerate(Target) :  ### 正確為哪類
                    if num == 1 :
                        matrix[k][j] += 1
                        break  # end of for k
                break  # end of for j

    AC_A = matrix[0][0] / test_size[0]
    AC_E = matrix[1][1] / test_size[1]
    AC_N = matrix[2][2] / test_size[2]
    AC_P = matrix[3][3] / test_size[3]
    AC_R = matrix[4][4] / test_size[4]
    print("\n-------------實驗結果-------------")
    print("各類句數")
    print("A:",test_size[0],"E:",test_size[1],"N:",test_size[2],"P:",test_size[3],"R:",test_size[4])
    print("總句數:",len(use_data))
    AC_UA = (AC_A + AC_E + AC_N + AC_P + AC_R) / 5
    AC_WA = (matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4]) / len(use_data)
    right = matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4]
    print('      A      E      N      P      R')
    print('A  %4d   %4d   %4d   %4d   %4d    ' % (matrix[0][0], matrix[0][1], matrix[0][2], matrix[0][3], matrix[0][4]))
    print('E  %4d   %4d   %4d   %4d   %4d    ' % (matrix[1][0], matrix[1][1], matrix[1][2], matrix[1][3], matrix[1][4]))
    print('N  %4d   %4d   %4d   %4d   %4d    ' % (matrix[2][0], matrix[2][1], matrix[2][2], matrix[2][3], matrix[2][4]))
    print('P  %4d   %4d   %4d   %4d   %4d    ' % (matrix[3][0], matrix[3][1], matrix[3][2], matrix[3][3], matrix[3][4]))
    print('R  %4d   %4d   %4d   %4d   %4d    ' % (matrix[4][0], matrix[4][1], matrix[4][2], matrix[4][3], matrix[4][4]))
    print('\nA: %f    E: %f     N: %f     P: %f     R: %f\n' % (AC_A*100, AC_E*100, AC_N*100, AC_P*100, AC_R*100))
    print("正確句數：",right)
    print("正確率:",AC_UA*100)
    print("=====================================================")

def tea_stu_model(data):
    
    model = Sequential()
    model.add(Dense(30, activation='sigmoid', input_shape=(384,)))
    model.add(Dense(5, activation='softmax'))

    model.load_weights("./model/ts_46_model.h5")
    #model.summary()
    result = model.predict(data)
    use_data = result
    AC_A = 0.0
    AC_E = 0.0
    AC_N = 0.0
    AC_P = 0.0
    AC_R = 0.0
    AC_WA = 0.0
    AC_UA = 0.0
    right=0#正確句數
    matrix = np.zeros((5, 5))

    use_data=result

    for i, (Target, Label) in enumerate( zip(test_label,use_data) ) :  ### target:答案 label:辨識結果
        m = np.max(Label)#辨識結果（最大類別）
        for j, value in enumerate(Label) :  ### 分到哪類
            if value == m :
                for k, num in enumerate(Target) :  ### 正確為哪類
                    if num == 1 :
                        matrix[k][j] += 1
                        break  # end of for k
                break  # end of for j

    AC_A = matrix[0][0] / test_size[0]
    AC_E = matrix[1][1] / test_size[1]
    AC_N = matrix[2][2] / test_size[2]
    AC_P = matrix[3][3] / test_size[3]
    AC_R = matrix[4][4] / test_size[4]
    print("\n-------------實驗結果-------------")
    print("各類句數")
    print("A:",test_size[0],"E:",test_size[1],"N:",test_size[2],"P:",test_size[3],"R:",test_size[4])
    print("總句數:",len(use_data))
    AC_UA = (AC_A + AC_E + AC_N + AC_P + AC_R) / 5
    AC_WA = (matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4]) / len(use_data)
    right = matrix[0][0] + matrix[1][1] + matrix[2][2] + matrix[3][3] + matrix[4][4]
    print('      A      E      N      P      R')
    print('A  %4d   %4d   %4d   %4d   %4d    ' % (matrix[0][0], matrix[0][1], matrix[0][2], matrix[0][3], matrix[0][4]))
    print('E  %4d   %4d   %4d   %4d   %4d    ' % (matrix[1][0], matrix[1][1], matrix[1][2], matrix[1][3], matrix[1][4]))
    print('N  %4d   %4d   %4d   %4d   %4d    ' % (matrix[2][0], matrix[2][1], matrix[2][2], matrix[2][3], matrix[2][4]))
    print('P  %4d   %4d   %4d   %4d   %4d    ' % (matrix[3][0], matrix[3][1], matrix[3][2], matrix[3][3], matrix[3][4]))
    print('R  %4d   %4d   %4d   %4d   %4d    ' % (matrix[4][0], matrix[4][1], matrix[4][2], matrix[4][3], matrix[4][4]))
    print('\nA: %f    E: %f     N: %f     P: %f     R: %f\n' % (AC_A*100, AC_E*100, AC_N*100, AC_P*100, AC_R*100))
    print("正確句數：",right)
    print("正確率:",AC_UA*100)
    print("=====================================================")
    for i, (Target, Label) in enumerate( zip(test_label,use_data) ) :  ### target:答案 label:辨識結果
        m = np.max(Label)#辨識結果（最大類別）
        for j, value in enumerate(Label) :  ### 分到哪類
            if value == m :
                if j == 0: # Anger
                    return [[1,0,0,0,0]]
                if j == 1: # Emphatic
                    return [[0,1,0,0,0]]
                if j == 2: # Neutral
                    return [[0,0,1,0,0]]
                if j == 3: # Positive
                    return [[0,0,0,1,0]]
                if j == 4: # Rest
                    return [[0,0,0,0,1]]


In [ ]:
for emo_class, emo_name in zip(combine, emotion):
    print ('Class: %s training procedure' % (emo_name))
    train_gan(emo_class, emo_name)
print ('\nEnd of training\n')


Class: Anger training procedure
Step 0, discriminator loss 1.54368 , generator loss 0.38689
INFO:tensorflow:./fake_model/Anger/model0.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:./fake_model/Anger/model1.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Step 2000, discriminator loss 1.16520 , generator loss 0.68738
INFO:tensorflow:./fake_model/Anger/model2.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:./fake_model/Anger/model3.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Step 4000, discriminator loss 1.14484 , generator loss 0.69222
INFO:tensorflow:./fake_model/Anger/model4.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:./fake_model/Anger/model5.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Step 6000, discriminator loss 1.04844 , generator loss 0.75291
INFO:tensorflow:./fake_model/Anger/model6.ckpt is not in all_model_checkpoint_paths. Manu

In [9]:
generate_fake_data()
add_data_amount = [0,0,0,0,0]
for i in range(5):
        fake_path = './fake_data/' + emotion[i] + '/' + fake_emotion_txt[i]
        gan_label = [[0,0,0,0,0]]
        gan_label[0][i] += 1
        gan_label = np.asarray(gan_label)
        print (fake_path)
        with open(fake_path) as f:
            for j in range(30):
                line = f.readline()
                if line == '': #EOF
                    break
                fake_data = []
                for numstr in line.split(' '):
                    if numstr:
                        numFloat = float(numstr)
                        fake_data.append(numFloat)
                fake_data = np.asarray(fake_data)
                fake_data = reshape(fake_data, [1,384])
                
                # run the knn procedure
                knn_label = knn(fake_data)
                knn_label = np.asarray(knn_label)
                # use teacher-student model to classify this fake data
                ts_label = tea_stu_model(fake_data)
                ts_label = np.asarray(ts_label)
                # whether to add this fake data to the training data
               
                print ('knn_label: {0} , ts_label: {1}'.format(knn_label, ts_label))
                if np.array_equal(gan_label, knn_label) == True and np.array_equal(knn_label, ts_label) == False:
                    print ('Add')
                    train_data = np.concatenate((train_data, fake_data), axis = 0)
                    train_label = np.concatenate((train_label, knn_label), axis = 0)
                    add_data_amount[i] += 1
                else:
                    print ('Discard')
        print ('%s extends %d data.' % (emotion[i], add_data_amount[i]))
np.savetxt('new_train_data.txt', train_data, delimiter=' ', fmt='%f')
np.savetxt('new_train_data_label.txt', train_label, delimiter=' ', fmt='%d')


End of generating.

./fake_data/Anger/fake_a.txt
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 0 1 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[1 0 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 1 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 1 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[1 0 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 1 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 0 1 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[1 0 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 0 1 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[1 0 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 1 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 1 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 0 1 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 1 0 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 1 0 0]]
Discard
knn_label: [[0 0 1 0 0]] , ts_label: [[0 0 1 0 0]]
Discard
knn_la

In [14]:
new_train_data = './new_train_data.txt'
new_train_data_label =  './new_train_data_label.txt'
fake_data,fake_label = read_fake_data(new_train_data,new_train_data_label)
print(fake_data[0][383])
print(fake_data.shape,fake_label.shape)
train_stmodel(fake_data,fake_label)


0.280944

((9981, 384), (9981, 5))
('\xe8\xa8\x93\xe7\xb7\xb4\xe8\xb3\x87\xe6\x96\x99\xef\xbc\x9a', (9981, 384), (9981, 5))
('\xe6\xb8\xac\xe8\xa9\xa6\xe8\xb3\x87\xe6\x96\x99\xef\xbc\x9a', (8257, 384), (8257, 5))
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_301 (Dense)            (None, 30)                11550     
_________________________________________________________________
dense_302 (Dense)            (None, 5)                 155       
Total params: 11,705.0
Trainable params: 11,705
Non-trainable params: 0.0
_________________________________________________________________
Epoch 1/600
1s - loss: 0.6531
Epoch 2/600
0s - loss: 0.6527
Epoch 3/600
0s - loss: 0.6525
Epoch 4/600
0s - loss: 0.6522
Epoch 5/600
0s - loss: 0.6522
Epoch 6/600
0s - loss: 0.6516
Epoch 7/600
0s - loss: 0.6512
Epoch 8/600
0s - loss: 0.6511
Epoch 9/600
0s - loss: 0.6510
Epoch 10/600
0s - loss: 0.6508
Epoch 11/600
0s 

In [19]:
tea_stu_model(fake_data)


-------------實驗結果-------------
各類句數
('A:', 611, 'E:', 1508, 'N:', 5377, 'P:', 215, 'R:', 546)
('\xe7\xb8\xbd\xe5\x8f\xa5\xe6\x95\xb8:', 9981)
      A      E      N      P      R
A    95    134    197    123     62    
E   247    346    462    291    162    
N   823   1255   1650   1069    580    
P    42     43     71     38     21    
R    77    114    189    112     54    

A: 15.548282    E: 22.944297     N: 30.686256     P: 17.674419     R: 9.890110

('\xe6\xad\xa3\xe7\xa2\xba\xe5\x8f\xa5\xe6\x95\xb8\xef\xbc\x9a', 2183.0)
('\xe6\xad\xa3\xe7\xa2\xba\xe7\x8e\x87:', 19.348672671890345)


[[0, 0, 0, 1, 0]]